# Astronomical Image Data Reduction
## for near-infrared wavelengths

Data reduction for near-infrared \(NIR\) images is similar to visible light images except for the added steps of removing the strong atmospheric background. Traditionally this was done using the software package [IRAF](http://iraf.noao.edu/) developed by the National Optical Astronomy Observatory \(NOAO\). This set of tools is a little buggy, and newer generations of astronomers are turning to Python instead.

In this tutorial, we will practice using the [AstroPy](http://docs.astropy.org/en/stable/index.html) package in a Python environment. First load the NumPy, Matplotlib Pyplot, and AstroPy FITS modules



In [1]:
import numpy as np #library for array operations and high-level math functions
import matplotlib.pyplot as plt #for making plots
from astropy.io import fits #to access FITS files

## Working with FITS files

FITS is the standard file format for astronomical imaging data. FITS files contain both multi-dimensional array data (1D spectra, 2D images, 3D+ data cubes) and descriptive information about the data.

To read in a FITS file use the `fits.open()` function where the argument is the filename. Use the code below to read in the 2MASS *J*-band image of HII region RCW 87. 

In [2]:
hdul = fits.open('RCW87_2MASS_J.fits')

fits.open() will return a list of top-level data objects called Header Data Unit (HDU) objects. To obtain a summary of the list type ``hdul.info()``

In [3]:
hdul.info()

Filename: RCW87_2MASS_J.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     113   (342, 342)   float32   


It is very common for a FITS file to contain only a single HDU called the PrimaryHDU which consists of header information and typically array data. If a HDU contains an array, the dimensions of the array are given.

The `.header` and `.data` attributes can be used to access the header and data portions of the HDU. Common practice is to separate the header and data portions. Execute the following.

In [4]:
hdr = hdul[0].header
data = hdul[0].data

To look at the entire header, type `print(repr(hdr))`

In [5]:
print(repr(hdr))

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                  -32 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                  342 / length of data axis 1                          
NAXIS2  =                  342 / length of data axis 2                          
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
ORDATE  = '990607  '             / Observation Ref Date (yymmdd)                
DAYNUM  = '0829    '             / Observation Day Num                          
FN_PRFX = 'j0829049'             / .rdo and .par filename prefix                
TYPE    = 'sci     '             / Scan type: dar flt sci cal tst               
SCANNO  =                   

On the left are the header keywords. There are just a few required keywords (SIMPLE, BITPIX, NAXIS, EXTEND), the rest are determined by the programs that write the FITS files. Thus, FITS files from different instruments typically have different sets of keywords. 

For example, in pipeline-reduced 2MASS images the keyword 'FILTER' contains the NIR filter or band observed in. To jview the value of only that keyword type

In [6]:
hdr['FILTER']

'j'

While most keywords are unique, the 'COMMENT' and 'HISTORY' keywords can appear multiple times in a header. When acessing them, a Python list is returned. Access both of these keywords below. 

In [7]:
hdr['HISTORY']

* Rdframe version= 980423     jhw.  Bgn Xeq = 01-11-13 00:52:54
* DFlat  vsn1.96 981203  Run on 11/13/01 at  0:50:19
> Image Created by pixphot - Version 010925p
>         Gene Kopan  IPAC/ Caltech

You can add a new history line with `hdr['HISTORY'] = 'contents of new line'`. Try it out and then access the 'HISTORY' keyword.

*warning*: the command `del hdr['HISTORY']` will delete the *entire* history list. To delete the last entry, type `hdr['HISTORY'][-1] = ''` 

## Displaying image data